# Simulations and moments

This script computes the simulations and associated moments of several version of the model, with and without self-fulfilling shocks, with short and long debt, and with the floating rate bond.

In [3]:
import Pkg; Pkg.activate(joinpath(@__DIR__, "..")); Pkg.instantiate()

using LTBonds
using Random 
using PrettyTables

  Activating project at `c:\Users\amado\Documents\GitHub\floating_rate`
    Updating registry at `C:\Users\amado\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Installed GR_jll ────────────── v0.66.2+0
   Installed StaticArraysCore ──── v1.1.0
   Installed Infiltrator ───────── v1.6.1
   Installed FastGaussQuadrature ─ v0.4.9
   Installed HTTP ──────────────── v1.2.1
   Installed ArrayInterfaceCore ── v0.1.17
   Installed NaNMath ───────────── v1.0.1
   Installed StaticArrays ──────── v1.5.4
   Installed Revise ────────────── v3.4.0
   Installed SnoopPrecompile ───── v1.0.0
   Installed StatsPlots ────────── v0.15.1
   Installed Plots ─────────────── v1.31.7
   Installed Setfield ──────────── v1.1.0
   Installed VectorizationBase ─── v0.21.46
   Installed JuliaInterpreter ──── v0.9.15
   Installed LoopVectorization ─── v0.12.122
   Installed ArrayInterface ────── v6.0.22
   Installed Distributions ─────── v0.25.67
   Installed GR ──

In [4]:
SAVE_MOMENTS = false # set to true to save the moments to file. 

false

In [5]:
benchmark_parameters =  let
    R = 1.01
    β = 0.9540232420
    pref = Preferences(β = β, u = make_CRRA(ra = 2))
    y = discretize(YProcess(n = 50, ρ = 0.948503, std = 0.027092, μ = 0.0, span = 3.0, tails = false))
    m = MTruncatedNormal(; std = 0.01, span = 2.0, quadN = 100)
    penalty = DefCosts(pen1 = -0.1881927550, pen2 = 0.2455843389, quadratic = true, reentry = 0.0385)
    η=0.1

    (R=R, pref=pref,  y=y, m=m, penalty=penalty, η=η )
end

(R = 1.01, pref = Preferences{CRRA2, Float64}(0.954023242, CRRA2), y = YDiscretized{YProcess{Float64, Int64}, Vector{Float64}, Matrix{Float64}, Float64}(YProcess{Float64, Int64}(50, 0.948503, 0.027092, 0.0, 3.0, false), [0.7736940032094284, 0.7818391711984078, 0.7900700885421625, 0.7983876579785925, 0.8067927917492923, 0.8152864116996035, 0.8238694493797185, 0.8325428461468501, 0.8413075532684764, 0.8501645320266717  …  1.176242906318547, 1.1886259621882678, 1.2011393823491128, 1.2137845392287432, 1.2265628197032372, 1.239475625249198, 1.2525243720974624, 1.2657104913884287, 1.279035429329019, 1.2925006473512937], [0.18120181525570228 0.12535504447193643 … 7.742222869464416e-72 1.1380889180175488e-74; 0.20275862347026066 0.16134314683122294 … 7.409028633939995e-69 1.2544237399520442e-71; … ; 0.0 0.0 … 0.1613431468312228 0.20275862347026077; 0.0 0.0 … 0.12535504447193643 0.18120181525570228], 1.0036640682963884), m = MTruncatedNormal{Float64, Distributions.Truncated{Distributions.Normal

In [20]:
mFR, mFRlowκ, mEGLT, mCKLT, mEGST, mCKST, mFRperp = let
    R, pref, y, m, penalty, η = benchmark_parameters

    bondFR = FloatingRateBond(;n = 350, min = 0.0, max = 1.5, λ = 0.05, κbar = 1.0) 
    bondFRlowκ = FloatingRateBond(;n = 350, min = 0.0, max = 1.5, λ = 0.05, κbar = 0.015) 
    bondLT = Bond(n = 350, min = 0.0, max = 1.5, κ = R - 1, λ = 0.05)  
    bondST = Bond(n = 350, min = 0.0, max = 1.5, κ = R - 1, λ = 1.0)  
    bondFRperp = FloatingRateBond(;n = 350, min = 0.0, max = 1.5, λ = 0.0, κbar = 0.05) 

    fr = CKLTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondFR, 
        def_costs = penalty, 
        R = R,
        η = η
    )   

    frlowκ = CKLTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondFRlowκ, 
        def_costs = penalty, 
        R = R,
        η = η
    )   

    egLT = LTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondLT, 
        def_costs = penalty, 
        R = R,
    )

    ckLT = CKLTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondLT, 
        def_costs = penalty, 
        R = R,
        η = η
    )

    egST = LTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondST, 
        def_costs = penalty, 
        R = R,
    )

    ckST = CKLTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondST, 
        def_costs = penalty, 
        R = R,
        η = η
    )

    frperp = CKLTBondModel(
        y = y,
        m = m, 
        preferences = pref, 
        bond = bondFRperp, 
        def_costs = penalty, 
        R = R,
        η = η
    )   


    (generate_workspace(fr), generate_workspace(frlowκ), generate_workspace(egLT), generate_workspace(ckLT), generate_workspace(egST), generate_workspace(ckST), generate_workspace(frperp) )
end;

In [21]:
for m ∈ (mFR, mFRlowκ, mEGST, mCKST, mEGLT, mCKLT, mFRperp)
    @time solve!(m; print_every = 100, max_iters = 5000)
end

1: (v = 1.4304966712931169, κ = 0.99, q = 0.6091402083520796, vD = 0.5336647908767169)
101: (v = 0.5946309463050952, κ = 0.8783079426374396, q = 0.8590067975969744, vD = 0.0016515421610243664)
201: (v = 0.5578743684368312, κ = 0.7464879959242077, q = 0.6999957288541196, vD = 0.00010840679993506797)
301: (v = 0.0006939164343044979, κ = 0.0005165681252381482, q = 7.086682121404397e-5, vD = 8.054937151058539e-6)
401: (v = 2.518262185446929e-7, κ = 1.3481843019746975e-7, q = 9.596437311465422e-9, vD = 9.789055255282619e-8)
501: (v = 2.0121184718391305e-9, κ = 9.525344957239668e-10, q = 5.3842819092153604e-11, vD = 9.34804234020703e-10)
566: (v = 9.85771464456775e-11, κ = 4.665134945014415e-11, q = 2.4933388687031766e-12, vD = 4.581579560181126e-11)
Converged.
1075.471720 seconds (36.55 k allocations: 3.350 MiB)
1: (v = 1.4304966712931169, κ = 0.0049999999999999906, q = 0.6091402083520796, vD = 0.5336647908767169)
101: (v = 0.006499144150186709, κ = 0.0014373646463945455, q = 0.033057665990

In [22]:
big_T = 20_000 
big_N = 1_000
rng = Random.seed!(1234)

TaskLocalRNG()

In [23]:
@time shocks, paths = create_shocks_paths(mCKST, big_T, big_N; rng);

  2.847760 seconds (65.02 k allocations: 2.824 GiB, 40.64% gc time)


In [24]:
##EGST
@time simulation!(paths, shocks, mEGST; n = big_T, trim = 1000, trim_def = 20)
@time moments_EGST = moments(paths, mEGST)

  0.460194 seconds (7.02 k allocations: 1.130 MiB)
  5.162148 seconds (5 allocations: 281.791 MiB, 40.69% gc time)


(mean_bp_y = 0.8205549556096196, mean_mv_y = 0.8200560615362792, mean_spread = 0.002623656025773225, std_spread = 0.003841584383537068, mean_κ = 0.010000000000000007, std_κ = 1.7347235229438105e-18, max_κ = 0.010000000000000009, std_c_y = 1.133027469576514, cor_tb_y = -0.2233765227992184, cor_r_y = -0.44506115122395706, cor_r_b_y = -0.2204906298011865, cor_r_tb = 0.852054829365093, def_rate = 0.0025025396737509142, run_share = 0.0)

In [25]:
##CKST
@time simulation!(paths, shocks, mCKST; n = big_T, trim = 1000, trim_def = 20)
@time moments_CKST = moments(paths, mCKST)

  0.242564 seconds (7.01 k allocations: 1.130 MiB)
  2.342926 seconds (5 allocations: 282.892 MiB, 15.45% gc time)


(mean_bp_y = 0.37518426640137736, mean_mv_y = 0.3749921881560013, mean_spread = 0.002271212205253653, std_spread = 0.0025437235941263786, mean_κ = 0.010000000000000007, std_κ = 1.7347235227610913e-18, max_κ = 0.010000000000000009, std_c_y = 1.054600263597107, cor_tb_y = -0.18708946710455696, cor_r_y = -0.6650009745665612, cor_r_b_y = -0.49312918202683675, cor_r_tb = 0.6858255794524165, def_rate = 0.0021505496569808047, run_share = 1.0)

In [26]:
##EGLT
@time simulation!(paths, shocks, mEGLT; n = big_T, trim = 1000, trim_def = 20)
@time moments_EGLT = moments(paths, mEGLT)

  0.219197 seconds (7.01 k allocations: 1.130 MiB)
  1.244674 seconds (5 allocations: 153.635 MiB, 1.40% gc time)


(mean_bp_y = 0.936006419319225, mean_mv_y = 0.7227623372721607, mean_spread = 0.07981495857468962, std_spread = 0.04431240237236437, mean_κ = 0.010000000000000007, std_κ = 1.7347235621220138e-18, max_κ = 0.010000000000000009, std_c_y = 1.1050229483901097, cor_tb_y = -0.39032161853095576, cor_r_y = -0.6475619630623043, cor_r_b_y = -0.027071889709726054, cor_r_tb = 0.7282486980693714, def_rate = 0.06712580452744432, run_share = 0.0)

In [27]:
##CKLT
@time simulation!(paths, shocks, mCKLT; n = big_T, trim = 1000, trim_def = 20)
@time moments_CKLT = moments(paths, mCKLT)

  0.228710 seconds (7.01 k allocations: 1.130 MiB)
  1.212637 seconds (5 allocations: 153.653 MiB, 0.97% gc time)


(mean_bp_y = 0.9358998257171705, mean_mv_y = 0.7226991030682838, mean_spread = 0.0798031593010469, std_spread = 0.044280324278591715, mean_κ = 0.010000000000000005, std_κ = 3.469447124224061e-18, max_κ = 0.010000000000000009, std_c_y = 1.105083694798529, cor_tb_y = -0.3903699194833873, cor_r_y = -0.6478664558235778, cor_r_b_y = -0.027875128139195328, cor_r_tb = 0.7283794525055333, def_rate = 0.06710014765866767, run_share = 0.0025512807429329525)

In [28]:
##FR
@time simulation!(paths, shocks, mFR; n = big_T, trim = 1000, trim_def = 20)
@time moments_FR = moments(paths, mFR)

  0.396365 seconds (7.01 k allocations: 1.130 MiB)
  2.009120 seconds (5 allocations: 281.944 MiB, 0.81% gc time)


(mean_bp_y = 0.7654853906270102, mean_mv_y = 0.765485390623778, mean_spread = 0.0025436925796378644, std_spread = 0.0035270310486864628, mean_κ = 0.01061557857523456, std_κ = 0.0008512388358720997, max_κ = 0.023665006159335755, std_c_y = 1.1061651695476236, cor_tb_y = -0.2162935945671702, cor_r_y = -0.4047458152305521, cor_r_b_y = -0.13794485153742567, cor_r_tb = 0.7403959365984468, def_rate = 0.0024301857845833164, run_share = 0.7687449968869519)

In [32]:
##FRlowκ
@time simulation!(paths, shocks, mFRlowκ; n = big_T, trim = 1000, trim_def = 20)
@time moments_FRlowκ = moments(paths, mFRlowκ)

  0.860725 seconds (7.01 k allocations: 1.129 MiB)
  2.332167 seconds (5 allocations: 206.471 MiB, 1.28% gc time)


(mean_bp_y = 0.8711728450206657, mean_mv_y = 0.7756564105049797, mean_spread = 0.03765188190172232, std_spread = 0.02872861655037881, mean_κ = 0.011107674734744053, std_κ = 0.0017222473400545508, max_κ = 0.015, std_c_y = 1.1184996886930902, cor_tb_y = -0.3041038688325507, cor_r_y = -0.43575629998683435, cor_r_b_y = 0.03485689410751214, cor_r_tb = 0.5653399043618292, def_rate = 0.0329320174746508, run_share = 0.0028570423897181637)

In [33]:
##FRperp
@time simulation!(paths, shocks, mFRperp; n = big_T, trim = 1000, trim_def = 20)
@time moments_FRperp = moments(paths, mFRperp)

  0.788531 seconds (7.02 k allocations: 1.130 MiB)
  2.876348 seconds (5 allocations: 277.494 MiB, 0.93% gc time)


(mean_bp_y = 0.7251413240530906, mean_mv_y = 0.7013185676057948, mean_spread = Inf, std_spread = NaN, mean_κ = 0.010980410406604668, std_κ = 0.017968214237910573, max_κ = 1.0, std_c_y = 1.1251307024519908, cor_tb_y = -0.21486120355582003, cor_r_y = NaN, cor_r_b_y = NaN, cor_r_tb = NaN, def_rate = 0.0038635447046659888, run_share = 0.6265333939118582)

In [34]:
pretty_table(
    [
        pairs(moments_EGST),
        pairs(moments_CKST),
        pairs(moments_EGLT),
        pairs(moments_CKLT),
        pairs(moments_FR),
        pairs(moments_FRlowκ),
        pairs(moments_FRperp)
    ],
    row_names = ["EGST", "CKST", "EGLT", "CKLT", "FR", "FRlowκ", "FRperpetuity"]
)

┌──────────────┬───────────┬───────────┬─────────────┬────────────┬───────────┬─────────────┬──────────┬─────────┬───────────┬───────────┬────────────┬──────────┬────────────┬────────────┐
│              │ mean_bp_y │ mean_mv_y │ mean_spread │ std_spread │    mean_κ │       std_κ │    max_κ │ std_c_y │  cor_tb_y │   cor_r_y │  cor_r_b_y │ cor_r_tb │   def_rate │  run_share │
├──────────────┼───────────┼───────────┼─────────────┼────────────┼───────────┼─────────────┼──────────┼─────────┼───────────┼───────────┼────────────┼──────────┼────────────┼────────────┤
│         EGST │  0.820555 │  0.820056 │  0.00262366 │ 0.00384158 │      0.01 │ 1.73472e-18 │     0.01 │ 1.13303 │ -0.223377 │ -0.445061 │  -0.220491 │ 0.852055 │ 0.00250254 │        0.0 │
│         CKST │  0.375184 │  0.374992 │  0.00227121 │ 0.00254372 │      0.01 │ 1.73472e-18 │     0.01 │  1.0546 │ -0.187089 │ -0.665001 │  -0.493129 │ 0.685826 │ 0.00215055 │        1.0 │
│         EGLT │  0.936006 │  0.722762 │    0.079815 │ 

In [15]:
SAVE_MOMENTS && open(joinpath(@__DIR__,"..","output","moments.txt"), "w") do f
    pretty_table(f,
    [
        pairs(moments_EGST),
        pairs(moments_CKST),
        pairs(moments_EGLT),
        pairs(moments_CKLT),
        pairs(moments_FR),
        pairs(moments_FRlowκ)
    ],
    row_names = ["EGST", "CKST", "EGLT", "CKLT", "FR", "FRlowκ"]
)
end

false

In [47]:
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1423
